In [1]:
import html
import sqlite3
import pandas as pd
import random
import csv
import requests
from html import unescape
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
session = requests.session()
from collections import Counter
from tqdm.auto import tqdm

In [2]:
conn = sqlite3.connect('newo4em.db')
cur = conn.cursor()

In [3]:
cur.execute("""
CREATE TABLE IF NOT EXISTS texts 
(id int PRIMARY KEY, title text, time_of_reading int, text text, category text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS tags 
(id int PRIMARY KEY, original text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS text_to_tag 
(id INTEGER PRIMARY KEY AUTOINCREMENT, id_text int, id_original int)
""")

conn.commit()

In [4]:
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
page_number = 1
url = f'https://newochem.ru/obshhestvo/page/{page_number}'
req = session.get(url, headers={'User-Agent': ua.random})

In [5]:
originals = ['The Verge', 'Los Angeles Times', 'Associated Press', 'Quartz', 
            'Foreign Policy', 'BBC Future', 'Jacobin Magazine', 'The Guardian', 
            'Priceonomics', 'Magazine', 'Citylab', 'The New York Times', 'Vice', 
            'Nautilus', 'The Atlantic', 'The Economist', 'Independent', 'Wired', 
             'Reader’s Digest', 'Medical News Today', 'Nautilus', 'Bloomberg', 'Vox', 
             'Slate Star Codex']
rubrics = ['obshhestvo', 'zdorove', 'kosmos', 'rossiya', 'politika',
            'igil', 'kultura', 'texnologii', 'lyudi', 'mir']

In [6]:
def parse_first_level_info(rubric, page_number):
    block = {}
    
    url = f'https://newochem.ru/{rubric}/page/{page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})       
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    href = soup.findall('h3', {'class':'post-title'})

    for l in href:
        links.append(l.prettify())
    for link in links:
        soup = BeautifulSoup(link, 'html.parser')
        block['href'] = soup.findall('a').get('href')
    
    req = session.get_all(block['href'], headers={'User-Agent': ua.random})       
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    block['title'] = soup.findall('h1', {'class': 'post-title'}).get_text()
    return block

In [7]:
def parse_second_level_info(block):
    url_one = block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = html.find('div', {'id': 'wtr-content'}).get_text().replace('\n', '')
    
    # проверяем, есть ли оригинал
    pattern = r'Оригинал.+?([\sA-Z\sa-z]+)</a>'
    if re.findall(pattern, page, flags= re.DOTALL) is not None:
        orig = re.findall(pattern, page, flags= re.DOTALL) 
        block['original'] = orig
    else:
        block['original'] = random.choice(originals)
        
    if block['original'] == '\n' or block['original'] == '\xa0 ' or block['original'] == '\n\t\t' or block['original'] == '\xa0Vox' or block['original'] == '\xa0':
            block['original'] = random.choice(originals)

    if isinstance(block['original'], list):
            block['original'] = block['original'][0]
        
    block['category'] = html.find('a', {'class': 'btn btn-cat'}).get_text()
    
    pattern_for_time = r' Среднее время на прочтение: ([\d]+).+ ([\d]+)'
    time = re.findall(pattern_for_time, reading_time, flags= re.DOTALL) 
    for e in time:
        minutes = int(e[0])*60
        seconds = minutes + int(e[1])
    block['reading_time'] = seconds

    return block

In [8]:
new_id = re.compile('https://newochem.ru/(.*?)$')

In [9]:
def get_nth_page(rubric, page_number):
    url = f'https://newochem.ru/{rubric}/page/{page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    blocks = []
    for rubric in rubrics:
        for page_number in range(100):
        # ошибку записываем в файл, если она возникает
            try:
                blocks.append(parse_first_level_info(rubric, page_number))
            except Exception as e:
                with open ("mistakes.txt", 'a', encoding="utf-8") as fh:
                    fh.write(str(e))
    return blocks

In [10]:
def write_to_db(block):
    print(block)
    origs = []
    authors = []
    for orig in block['originals']:
        # если такой тег уже видели, то запоминаем его айди
        if orig in db_tags:
            origs.append(db_tags[orig])
        else:
            db_tags[orig] = len(db_tags) + 1 
            cur.execute('INSERT INTO tags VALUES (?, ?)', (len(db_tags), orig))
            conn.commit()
            origs.append(db_tags[orig])

    text_id = len(seen_news) + 1
    cur.execute(
        'INSERT INTO texts VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
        (int(text_id), block['title'], block['pub_day'], block['title'], 
         block['time'], block['full_text'], block['original']))
    #заполняем таблицу связей
    origs = [(text_id, t) for t in tags]
    cur.executemany(
        'INSERT INTO tag_and_text (id_text, id_tag) VALUES (?, ?)', origs)
    conn.commit()

In [11]:
conn = sqlite3.connect('newo4em.db')
cur = conn.cursor()
cur.execute('SELECT original, id FROM tags')
conn.commit()
db_tags = {}
for name, idx in cur.fetchall():
    db_tags[name] = idx

conn.commit()

In [ ]:
def run_all(pages):
    for rubric in rubrics:
        for i in tqdm(range(pages)):
            blocks = get_nth_page(rubric, i+1)
            for block in blocks:
                write_to_db(block)
run_all(16)